<a href="https://colab.research.google.com/github/LDwise/autoWiseman/blob/main/WisemanClear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# coding=utf-8
Class = "4b" #@param ["4a", "4b", "4c", "4d", "5a", "5b", "5c", "5d", "6a", "6b", "6c", "6d"]
Class_number = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31" , "32", "33", "34", "35", "36", "37", "38", "39", "40"]
from google.colab import output
from time import time
try:
	from selenium import webdriver
except:
	# 清除輸出Log
	!apt -q -qq update
	!apt -q -qq install chromium-chromedriver
	!pip -q install selenium
	from selenium import webdriver
	output.clear()
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# 設置Chrome啓動設置
options = webdriver.ChromeOptions()
# 無界面運行
options.add_argument('--headless')
# 以最高权限运行
options.add_argument('--no-sandbox')
# 忽略警告\錯誤\異常
options.add_argument('log-level=3')
# log-level:
# Sets the minimum log level.
# Valid values are from 0 to 3:
#	 INFO = 0,
#	 WARNING = 1,
#	 LOG_ERROR = 2,
#	 LOG_FATAL = 3.
# default is 0.
# 避免崩潰
options.add_argument('--disable-dev-shm-usage')
# 開啓Chrome並加入基礎設置
driver = webdriver.Chrome(options=options)
# 設定等待元素時間上限(單位秒)
wait = WebDriverWait(driver,10)
for Class_number in Class_number:
	# 登出Wiseman
	driver.get('https://lms.wiseman.com.hk/lms/user/logout.do')
	# 在此修改你的賬號(小寫) e.g.5a30
	information=Class+Class_number
	# 登入Wiseman
	driver.get('https://lms.wiseman.com.hk/lms/user/login.do?username=tiaccwhf.%s&password=2020'%information)
	# 等待頁面加載完畢
	try:
		wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="card"]/div[@class="card-body"]/div[@class="text-right"]/h1[@class="font-light mb-0 d-inline-block d-md-block"]')))
	except:
		continue
	student_name=str(driver.find_element_by_xpath('//div[@class="profile-text"]/a[@class="dropdown-toggle link u-dropdown"]').get_attribute('textContent'))#//*[@id="main-wrapper"]/aside/div/div[1]/div/div[2]/a
	before_score=int(str(driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]/h1').get_attribute('textContent')).replace('&nbsp;',''))#//*[@id="main-wrapper"]/div/div/div[2]/div[1]/div/div/div/h1
	print('Welcome!%s\nYour Score:%d'%(student_name,before_score))
	# 進入課程列表
	driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/lessons.shtml')
	# 獲取所有鏈接和標題
	uncomplete=[]
	print('開始捕獲所有課程',end='\r')
	for a in driver.find_elements_by_xpath('//a[contains(@href,"/lms/user/secure/course/eb/select_theme/selectLesson")]'):
		# 獲取標題
		title=a.get_attribute('textContent')
		# 獲取鏈接
		link=a.get_attribute('href')
		# 加入列表
		uncomplete.append({'title':title,'link':link})
	lessons_length=len(uncomplete)
	print('課程總數:%d'%lessons_length)
	now_lessons=0
	# 在列表範圍内循環
	for i in uncomplete:
		now_lessons+=1
		print('清除課程[%d/%d]:%s(%s)'%(now_lessons,lessons_length,i['title'],i['link']))
		# 進入鏈接
		driver.get(i['link'])
		wait.until(EC.presence_of_element_located((By.XPATH,'//iframe[@id="course"]')))
		# Loading time
		driver.switch_to.frame('course')
		# 等待頁面加載完畢
		wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="head course-head"]/div[@class="course-head-text"]')))
		# wait.until(EC.presence_of_element_located((By.XPATH,'//label[@water-level]')))
		# Return top
		driver.switch_to.default_content()
		wait.until(EC.presence_of_element_located((By.XPATH,'//iframe[@id="course"]')))
		driver.execute_script('''
doCourseAPIKeepAlive=function(){return false;};
API.cmiData["cmi.core.lesson_status"]="new";
var formData = new FormData();
formData.set('token', API.token);
formData.set('data', API.cmiData.toJSONString());
navigator.sendBeacon("commit.do", formData);
// API.cmiData["cmi.core.score.raw"]=100;
// API.cmiData["cmi.core.lesson_status"]="completed";
// navigator.sendBeacon("commit.do", formData);
navigator.sendBeacon=function(){return false;};
		''')
	# 進入分數頁面
	driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/dashboard.shtml')
	# 等待頁面加載完畢
	wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="card-body"]/div[@class="text-right"]/h1')))
	# 完成后的分數
	after_score=int(str(driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]/h1').get_attribute('textContent')).replace('&nbsp;',''))

	print('%s[%d=>%d]'%(student_name,before_score,after_score))
driver.quit()